# 🔒 Differential Privacy: Privacy-Preserving Machine Learning

**Core Concept**: Differential privacy provides mathematical guarantees that machine learning models trained on sensitive data don't leak information about individual training examples.

## 🎯 Privacy Through Noise
1.  **The Problem**: ML models can memorize training data (privacy leak)
2.  **The Solution**: Add calibrated noise during training
3.  **The Guarantee**: If algorithm runs on two datasets differing by one record, outputs look nearly identical
4.  **The Result**: Attacker can't determine if individual's data was in training set

## 📊 The Privacy-Utility Tradeoff
-   **More privacy (lower ε)**: More noise → Lower model accuracy
-   **Less privacy (higher ε)**: Less noise → Higher model accuracy
-   **Optimal range**: ε between 0.5 and 10.0 depending on sensitivity

## 🌍 Real-World Examples
-   **Apple**: Emoji usage, Safari browsing, Health app data
-   **US Census Bureau**: Population statistics with formal privacy guarantees
-   **Google**: Federated learning with differential privacy
-   **Healthcare**: Medical research without exposing patient data

This notebook demonstrates both the **Laplace Mechanism** (for private statistics) and **DP-SGD** (for private model training).

## 🛠️ Step 1: Setup & Data Loading

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd

# Set random seeds
np.random.seed(42)
tf.random.set_seed(42)

# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

# Normalize and reshape
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

# Convert labels to categorical
y_train_cat = keras.utils.to_categorical(y_train, 10)
y_test_cat = keras.utils.to_categorical(y_test, 10)

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")
print("\n✅ Data loaded successfully!")

## 🔢 Step 2: The Laplace Mechanism - Private Statistics

Before training models, let's understand differential privacy through a simple example: computing the average age in a dataset.

In [ ]:
def laplace_mechanism(true_value, sensitivity, epsilon):
    """
    Add Laplace noise to achieve differential privacy.
    
    Args:
        true_value: The actual computed value
        sensitivity: Maximum change in value from adding/removing one record
        epsilon: Privacy budget (lower = more privacy)
    
    Returns:
        Noisy value with ε-DP guarantee
    """
    scale = sensitivity / epsilon
    noise = np.random.laplace(loc=0, scale=scale)
    return true_value + noise

# Example: Private average age
# Simulate age data
ages = np.random.randint(18, 80, size=1000)
true_avg_age = np.mean(ages)

# Sensitivity: (max_age - min_age) / n = (80 - 18) / 1000 = 0.062
sensitivity = (80 - 18) / len(ages)

# Test different epsilon values
epsilons = [0.1, 0.5, 1.0, 5.0, 10.0]

print(f"True average age: {true_avg_age:.2f} years\n")
print("Privacy Budget (ε) | Private Average | Error | Privacy Level")
print("-" * 70)

for eps in epsilons:
    private_avg = laplace_mechanism(true_avg_age, sensitivity, eps)
    error = abs(private_avg - true_avg_age)
    
    if eps < 1.0:
        privacy = "Very Strong"
    elif eps < 5.0:
        privacy = "Strong"
    else:
        privacy = "Moderate"
    
    print(f"ε = {eps:5.1f}          | {private_avg:14.2f} | {error:5.2f} | {privacy}")

print("\n💡 Notice: Lower epsilon (stronger privacy) → Larger error")

## 📊 Step 3: Visualize Laplace Noise Distribution

In [ ]:
# Generate many private averages to see noise distribution
num_samples = 1000
epsilon_test = 1.0

private_averages = [laplace_mechanism(true_avg_age, sensitivity, epsilon_test) 
                   for _ in range(num_samples)]

# Plot distribution
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(private_averages, bins=50, alpha=0.7, edgecolor='black')
plt.axvline(true_avg_age, color='red', linestyle='--', linewidth=2, label='True Value')
plt.xlabel('Private Average Age', fontsize=11)
plt.ylabel('Frequency', fontsize=11)
plt.title(f'Distribution of Private Averages (ε={epsilon_test})', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)

# Compare different epsilons
plt.subplot(1, 2, 2)
for eps in [0.5, 1.0, 5.0]:
    samples = [laplace_mechanism(true_avg_age, sensitivity, eps) for _ in range(1000)]
    plt.hist(samples, bins=50, alpha=0.5, label=f'ε={eps}')
plt.axvline(true_avg_age, color='red', linestyle='--', linewidth=2, label='True Value')
plt.xlabel('Private Average Age', fontsize=11)
plt.ylabel('Frequency', fontsize=11)
plt.title('Noise Distribution Across Different ε Values', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Visualization shows:")
print("   - True value is hidden in a 'cloud' of noise")
print("   - Smaller epsilon → Wider noise distribution → Stronger privacy")

## 🏗️ Step 4: Train Baseline Model (No Privacy)

In [ ]:
def create_model():
    """Create a simple CNN for MNIST."""
    model = keras.Sequential([
        layers.Conv2D(16, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    return model

# Create and train baseline model
print("Training baseline model (no privacy)...\n")

baseline_model = create_model()
baseline_model.compile(optimizer='adam', 
                      loss='categorical_crossentropy', 
                      metrics=['accuracy'])

history_baseline = baseline_model.fit(
    X_train, y_train_cat,
    epochs=3,
    batch_size=256,
    validation_split=0.1,
    verbose=1
)

# Evaluate
test_loss, baseline_acc = baseline_model.evaluate(X_test, y_test_cat, verbose=0)
print(f"\n✅ Baseline Model Accuracy: {baseline_acc*100:.2f}%")

## 🔐 Step 5: Implement DP-SGD Components

DP-SGD has three key steps:
1.  Compute per-sample gradients (not averaged)
2.  Clip each gradient to bounded L2 norm
3.  Add calibrated noise to the average

In [ ]:
def clip_gradients(gradients, l2_norm_clip):
    """
    Clip gradients to maximum L2 norm.
    
    Args:
        gradients: List of gradient tensors
        l2_norm_clip: Maximum allowed L2 norm
    
    Returns:
        Clipped gradients
    """
    clipped = []
    for grad in gradients:
        if grad is not None:
            grad_norm = tf.norm(grad)
            # Clip factor: min(1.0, clip_norm / actual_norm)
            clip_factor = tf.minimum(1.0, l2_norm_clip / (grad_norm + 1e-10))
            clipped.append(grad * clip_factor)
        else:
            clipped.append(None)
    return clipped

def add_noise_to_gradients(gradients, noise_multiplier, l2_norm_clip):
    """
    Add Gaussian noise to gradients for differential privacy.
    
    Args:
        gradients: List of gradient tensors
        noise_multiplier: Scale of noise relative to clipping norm
        l2_norm_clip: Gradient clipping norm
    
    Returns:
        Noisy gradients
    """
    noise_stddev = noise_multiplier * l2_norm_clip
    
    noisy = []
    for grad in gradients:
        if grad is not None:
            noise = tf.random.normal(tf.shape(grad), mean=0.0, stddev=noise_stddev)
            noisy.append(grad + noise)
        else:
            noisy.append(None)
    return noisy

print("✅ DP-SGD helper functions defined:")
print("   - clip_gradients(): Bounds gradient norms")
print("   - add_noise_to_gradients(): Adds calibrated Gaussian noise")

## 🛡️ Step 6: Train Model with DP-SGD

Now we train a model with differential privacy guarantees.

In [ ]:
def train_dp_model(X_train, y_train, epochs=3, batch_size=256, 
                   l2_norm_clip=1.0, noise_multiplier=1.1):
    """
    Train a model with DP-SGD.
    
    Args:
        X_train, y_train: Training data
        epochs: Number of training epochs
        batch_size: Batch size
        l2_norm_clip: Gradient clipping norm
        noise_multiplier: Noise scale parameter
    
    Returns:
        Trained model
    """
    model = create_model()
    optimizer = keras.optimizers.Adam(learning_rate=0.001)
    loss_fn = keras.losses.CategoricalCrossentropy()
    
    # Prepare dataset
    dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    dataset = dataset.shuffle(10000).batch(batch_size)
    
    print(f"Training with DP-SGD:")
    print(f"  L2 norm clip: {l2_norm_clip}")
    print(f"  Noise multiplier: {noise_multiplier}")
    print(f"  Batch size: {batch_size}\n")
    
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        epoch_loss = []
        epoch_acc = []
        
        for step, (batch_x, batch_y) in enumerate(dataset):
            with tf.GradientTape() as tape:
                predictions = model(batch_x, training=True)
                loss = loss_fn(batch_y, predictions)
            
            # Compute gradients
            gradients = tape.gradient(loss, model.trainable_variables)
            
            # DP-SGD modifications
            # 1. Clip gradients
            clipped_gradients = clip_gradients(gradients, l2_norm_clip)
            
            # 2. Add noise
            noisy_gradients = add_noise_to_gradients(clipped_gradients, 
                                                    noise_multiplier, 
                                                    l2_norm_clip)
            
            # 3. Apply gradients
            optimizer.apply_gradients(zip(noisy_gradients, model.trainable_variables))
            
            # Track metrics
            epoch_loss.append(loss.numpy())
            acc = tf.reduce_mean(tf.cast(
                tf.equal(tf.argmax(predictions, axis=1), tf.argmax(batch_y, axis=1)),
                tf.float32
            ))
            epoch_acc.append(acc.numpy())
            
            if step % 50 == 0:
                print(f"  Step {step}: Loss={loss.numpy():.4f}, Acc={acc.numpy():.4f}")
        
        print(f"  Epoch {epoch + 1} - Avg Loss: {np.mean(epoch_loss):.4f}, Avg Acc: {np.mean(epoch_acc):.4f}\n")
    
    return model

# Train DP model with moderate privacy (epsilon ~ 1.0)
dp_model = train_dp_model(X_train, y_train_cat, 
                         epochs=3, 
                         batch_size=256, 
                         l2_norm_clip=1.0, 
                         noise_multiplier=1.1)

# Evaluate
dp_loss, dp_acc = dp_model.evaluate(X_test, y_test_cat, verbose=0)
print(f"\n✅ DP Model Accuracy: {dp_acc*100:.2f}%")
print(f"Baseline Accuracy: {baseline_acc*100:.2f}%")
print(f"Accuracy Gap: {(baseline_acc - dp_acc)*100:.2f}%")

## 📐 Step 7: Privacy Budget Calculation (Simplified)

Calculate the privacy budget (epsilon) spent during training.

In [ ]:
def calculate_epsilon_simple(steps, noise_multiplier, batch_size, dataset_size, delta=1e-5):
    """
    Simplified epsilon calculation using strong composition.
    
    Note: Real implementations should use RDP accountant for accuracy.
    This is an approximation for demonstration.
    """
    # Sampling probability
    q = batch_size / dataset_size
    
    # Per-step epsilon (simplified)
    # Real formula involves complex privacy accounting
    epsilon_per_step = q / noise_multiplier
    
    # Total epsilon with advanced composition (approximate)
    epsilon_total = epsilon_per_step * np.sqrt(2 * steps * np.log(1 / delta))
    
    return epsilon_total

# Calculate privacy budget for our DP model
num_epochs = 3
batch_size = 256
dataset_size = len(X_train)
steps_per_epoch = dataset_size // batch_size
total_steps = num_epochs * steps_per_epoch
noise_multiplier = 1.1
delta = 1e-5

epsilon = calculate_epsilon_simple(total_steps, noise_multiplier, 
                                  batch_size, dataset_size, delta)

print("="*60)
print("PRIVACY BUDGET ANALYSIS")
print("="*60)
print(f"Total training steps:     {total_steps}")
print(f"Noise multiplier:         {noise_multiplier}")
print(f"Gradient clipping norm:   1.0")
print(f"Delta (δ):                {delta}")
print(f"\nPrivacy Budget (ε):       {epsilon:.2f}")
print("="*60)

if epsilon < 1.0:
    privacy_level = "Very Strong Privacy"
elif epsilon < 5.0:
    privacy_level = "Strong Privacy"
elif epsilon < 10.0:
    privacy_level = "Moderate Privacy"
else:
    privacy_level = "Weak Privacy"

print(f"\nPrivacy Level: {privacy_level}")
print(f"\n💡 Interpretation: With ε={epsilon:.2f}, the model satisfies")
print(f"   ({epsilon:.2f}, {delta})-differential privacy.")

## 🔬 Step 8: Privacy-Utility Tradeoff Analysis

Train models with different epsilon values to see the tradeoff.

In [ ]:
print("Exploring privacy-utility tradeoff...\n")

# Test different noise multipliers (inverse relationship with epsilon)
noise_multipliers = [0.5, 0.8, 1.1, 1.5, 2.0]
results = []

for nm in noise_multipliers:
    print(f"Training with noise multiplier = {nm}...")
    
    # Train model
    model_temp = train_dp_model(X_train[:30000], y_train_cat[:30000],  # Use subset for speed
                               epochs=2, 
                               batch_size=256, 
                               l2_norm_clip=1.0, 
                               noise_multiplier=nm)
    
    # Evaluate
    _, acc = model_temp.evaluate(X_test, y_test_cat, verbose=0)
    
    # Calculate epsilon
    steps = 2 * (30000 // 256)
    eps = calculate_epsilon_simple(steps, nm, 256, 30000, delta=1e-5)
    
    results.append({
        'Noise Multiplier': nm,
        'Epsilon': eps,
        'Accuracy': acc * 100
    })
    
    print(f"  Epsilon: {eps:.2f}, Accuracy: {acc*100:.2f}%\n")

# Create results table
df_results = pd.DataFrame(results)
print("\n" + "="*60)
print("PRIVACY-UTILITY TRADEOFF")
print("="*60)
print(df_results.to_string(index=False))
print("="*60)

## 📊 Step 9: Visualize Privacy-Utility Tradeoff

In [ ]:
# Plot tradeoff curve
fig, ax = plt.subplots(figsize=(10, 6))

epsilons = df_results['Epsilon'].values
accuracies = df_results['Accuracy'].values

# Add baseline (no privacy)
epsilons_plot = np.append(epsilons, [100])  # Represent infinity as 100 for plotting
accuracies_plot = np.append(accuracies, [baseline_acc * 100])

ax.plot(epsilons_plot[:-1], accuracies_plot[:-1], 'o-', linewidth=2, 
        markersize=8, label='DP Models', color='blue')
ax.scatter([epsilons_plot[-1]], [accuracies_plot[-1]], 
          s=100, color='red', marker='*', label='No Privacy (Baseline)', zorder=5)

ax.set_xlabel('Privacy Budget (ε)', fontsize=12)
ax.set_ylabel('Test Accuracy (%)', fontsize=12)
ax.set_title('Privacy-Utility Tradeoff Curve', fontsize=14)
ax.grid(True, alpha=0.3)
ax.legend(fontsize=11)

# Annotate privacy zones
ax.axvspan(0, 1, alpha=0.1, color='green', label='Strong Privacy')
ax.axvspan(1, 5, alpha=0.1, color='yellow')
ax.axvspan(5, 10, alpha=0.1, color='orange')

# Add text annotations
ax.text(0.5, accuracies_plot.min() + 1, 'Strong\nPrivacy', 
       ha='center', fontsize=9, color='darkgreen')
ax.text(3, accuracies_plot.min() + 1, 'Moderate\nPrivacy', 
       ha='center', fontsize=9, color='darkorange')
ax.text(7.5, accuracies_plot.min() + 1, 'Weak\nPrivacy', 
       ha='center', fontsize=9, color='darkred')

plt.tight_layout()
plt.show()

print("\n💡 Key Insight:")
print("   Lower epsilon (stronger privacy) → Lower accuracy")
print("   The optimal epsilon depends on your privacy requirements!")

## 🕵️ Step 10: Membership Inference Attack Test

Test if an attacker can determine if a specific example was in the training set.

In [ ]:
def membership_inference_attack(model, train_data, train_labels, test_data, test_labels):
    """
    Simple membership inference attack based on prediction confidence.
    
    Returns:
        Attack accuracy (50% = random guessing = perfect privacy)
    """
    # Get prediction confidences
    train_preds = model.predict(train_data, verbose=0)
    test_preds = model.predict(test_data, verbose=0)
    
    # Extract confidence for correct class
    train_confidences = [train_preds[i, train_labels[i]] for i in range(len(train_labels))]
    test_confidences = [test_preds[i, test_labels[i]] for i in range(len(test_labels))]
    
    # Threshold-based attack: high confidence → likely in training set
    threshold = np.median(train_confidences + test_confidences)
    
    # Classify based on threshold
    train_predictions = np.array(train_confidences) > threshold  # Should be True
    test_predictions = np.array(test_confidences) > threshold    # Should be False
    
    # Calculate attack accuracy
    train_correct = np.mean(train_predictions)
    test_correct = np.mean(~test_predictions)
    attack_accuracy = (train_correct + test_correct) / 2
    
    return attack_accuracy * 100

# Test on baseline model (no privacy)
print("Testing membership inference attack...\n")

train_subset = X_train[:1000]
train_labels_subset = y_train[:1000]
test_subset = X_test[:1000]
test_labels_subset = y_test[:1000]

baseline_attack_acc = membership_inference_attack(
    baseline_model, train_subset, train_labels_subset, 
    test_subset, test_labels_subset
)

dp_attack_acc = membership_inference_attack(
    dp_model, train_subset, train_labels_subset, 
    test_subset, test_labels_subset
)

print("="*60)
print("MEMBERSHIP INFERENCE ATTACK RESULTS")
print("="*60)
print(f"Baseline Model (No Privacy):  {baseline_attack_acc:.1f}%")
print(f"DP Model (ε≈{epsilon:.1f}):            {dp_attack_acc:.1f}%")
print(f"Random Guessing (Perfect):    50.0%")
print("="*60)

print("\n💡 Interpretation:")
print("   - High attack accuracy (>60%) = Privacy leak")
print("   - Near 50% = Strong privacy (attacker can't do better than random)")

if dp_attack_acc < baseline_attack_acc - 5:
    print("\n✅ Differential privacy successfully reduced privacy leakage!")
else:
    print("\n⚠️ Consider increasing noise multiplier for stronger privacy.")

## 📝 Summary

### What We Demonstrated:
✅ **Laplace Mechanism**: Adding calibrated noise to statistics for privacy  
✅ **DP-SGD**: Training neural networks with formal privacy guarantees  
✅ **Privacy Budget**: Epsilon quantifies privacy loss (lower = stronger privacy)  
✅ **Privacy-Utility Tradeoff**: More privacy → More noise → Lower accuracy  
✅ **Membership Inference**: DP models resist privacy attacks  

### Key Concepts:

#### Differential Privacy Definition
An algorithm M satisfies **ε-differential privacy** if for any two datasets D1, D2 differing by one record:
```
Pr[M(D1) = o] ≤ e^ε × Pr[M(D2) = o]
```
**Meaning**: Outputs are nearly identical whether your data is included or not.

#### DP-SGD Algorithm
```python
# Standard SGD
gradients = compute_gradients(batch)
apply_gradients(gradients)

# DP-SGD
per_sample_grads = compute_per_sample_gradients(batch)
clipped_grads = clip_gradients(per_sample_grads, norm=1.0)
noisy_grads = add_gaussian_noise(clipped_grads, scale=noise_multiplier)
apply_gradients(noisy_grads)
```

### Privacy Parameters:

#### Epsilon (ε) - Privacy Budget
-   **ε < 1.0**: Very strong privacy, significant accuracy loss
-   **ε ≈ 1.0**: Strong privacy, moderate accuracy loss (~5-10%)
-   **ε = 5.0**: Moderate privacy, small accuracy loss (~2-5%)
-   **ε > 10.0**: Weak privacy, minimal accuracy loss

#### Delta (δ) - Failure Probability
-   Typical value: **1/n²** where n is dataset size
-   For 10,000 samples: δ = 1e-5
-   Privacy guarantee holds "except with probability δ"

#### Noise Multiplier
-   Controls amount of noise added to gradients
-   Higher value → Lower epsilon → Stronger privacy
-   Typical range: 0.5 to 2.0

#### Gradient Clipping Norm
-   Bounds influence of any single training example
-   Typical value: 0.5 to 1.5
-   Too low: slow learning; too high: weak privacy

### Real-World Applications:

#### Apple's Differential Privacy
-   Emoji usage patterns
-   Safari browsing data
-   Health app statistics
-   **Method**: Local differential privacy (noise added on-device)

#### US Census Bureau
-   2020 Census data release
-   Population counts by demographics
-   **Goal**: Protect individual responses while releasing aggregate statistics

#### Healthcare Research
-   Clinical trial results
-   Epidemic modeling
-   Medical image analysis
-   **Requirement**: HIPAA compliance + research utility

### The Fundamental Tradeoff:
```
Privacy ↔ Utility

Stronger Privacy (lower ε):       Weaker Privacy (higher ε):
  + Better protection                + Higher accuracy
  + Attack resistance                + Faster training
  - Lower accuracy                   - Privacy leakage risk
  - More noise                       - Less noise
```

### When to Use Differential Privacy:
✅ Training on sensitive personal data (health, finance, personal info)  
✅ Regulatory requirements (GDPR, HIPAA, CCPA)  
✅ Public data releases (census, research datasets)  
✅ Federated learning scenarios  
✅ When formal privacy guarantees are required  

### Best Practices:
1.  **Set privacy budget (epsilon) before training** based on sensitivity
2.  **Use established libraries** (TensorFlow Privacy, Opacus) for correct implementation
3.  **Monitor epsilon during training** and stop when budget is exhausted
4.  **Test with membership inference** to verify privacy guarantees
5.  **Balance privacy and utility** based on application requirements
6.  **Increase dataset size** when possible (larger n → better tradeoff)
7.  **Don't reuse privacy budget** across multiple experiments

**Differential privacy is the gold standard for privacy-preserving machine learning with mathematical guarantees.**